In [1]:
using TaylorSeries, TaylorIntegration
using Plots, LaTeXStrings
# plotly()
pyplot(grid = :false, legend = :false, size=(600,600), color=:black, markersize = 0.3 )
const Z = 2.0
const order = 20
const t0 = 0.0
const t_max = 0.5 
const abs_tol = 1.0E-20
const steps = 500_000

500000

In [17]:
doc"""
    jetcoeffs!(f, t, x)
Specialized method of `jetcoeffs!` for jet transport applications.
Returns an updated `x` using the recursion relation of the
derivatives from the ODE $\dot{x}=dx/dt=f(t,x)$.
`f` is the function defining the RHS of the ODE, `x` is a `Taylor1{TaylorN{T}}`
or a vector of that type, containing the Taylor expansion
of the dependent variables of the ODE and `t` is the independent
variable of type `T`.
Initially, `x` contains only the 0-th order Taylor coefficients of
the current system state (the initial conditions), and `jetcoeffs!`
computes recursively the high-order derivates back into `x`.
"""
function jetcoeffs!{T<:Number}(eqsdiff, t0::T, x::Taylor1{TaylorN{T}})
    order = x.order
    for ord in 1:order
        ordnext = ord+1

        # Set `xaux`, auxiliary Taylor1 variable to order `ord`
        @inbounds xaux = Taylor1( x.coeffs[1:ord] )

        # Equations of motion
        # TODO! define a macro to optimize the eqsdiff
        xdot = eqsdiff(t0, xaux)

        # Recursion relation
        @inbounds x.coeffs[ordnext] = xdot.coeffs[ord]/ord
    end
    nothing
end

function jetcoeffs!{T<:Number}(eqsdiff, t0::T, x::Vector{Taylor1{TaylorN{T}}})
    order = x[1].order
    xaux = similar(x)
    for ord in 1:order
        ordnext = ord+1
        @show ord
        # Set `xaux`, auxiliary vector of Taylor1 to order `ord`
        @inbounds for j in eachindex(x)
            a1 = Taylor1( x[j].coeffs[1:ord] )
            @show a1
            xaux[j] = a1
        end
        
        @show xaux
        # Equations of motion
        # TODO! define a macro to optimize the eqsdiff
        xdot = eqsdiff(t0, xaux)

        # Recursion relations
        @inbounds for j in eachindex(x)
            x[j].coeffs[ordnext] = xdot[j].coeffs[ord]/ord
        end
    end
    nothing
end

# stepsize
doc"""
    stepsize(x, epsilon)
Specialized method of `stepsize` for jet transport applications. Returns a
time-step for a `x::Taylor1{TaylorN{T}}` using a prescribed absolute tolerance
`epsilon::T`, for `T<:Number`.
"""
function stepsize{T<:Number}(x::Taylor1{TaylorN{T}}, epsilon::T)
    ord = x.order
    h = T(Inf)
    for k in (ord-1, ord)
        @inbounds aux = Array{T}(x.coeffs[k+1].order)
        for i in 1:x.coeffs[k+1].order
            @inbounds aux[i] = norm(x.coeffs[k+1].coeffs[i].coeffs,Inf)
        end
        aux == zeros(T, length(aux)) && continue
        aux = epsilon ./ aux
        kinv = one(T)/k
        aux = aux.^kinv
        h = min(h, minimum(aux))
    end
    return h
end

doc"""
    stepsize(q, epsilon)
Specialized method of `stepsize` for jet transport applications. Returns the
minimum time-step for `q::Array{Taylor1{TaylorN{T}},1}`, using a prescribed absolute
tolerance `epsilon::T`, for `T<:Number`.
"""
function stepsize{T<:Number}(q::Array{Taylor1{TaylorN{T}},1}, epsilon::T)
    h = T(Inf)
    for i in eachindex(q)
        @inbounds hi = stepsize( q[i], epsilon )
        h = min( h, hi )
    end
    return h
end

# taylorstep and taylorstep!
doc"""
    taylorstep(f, t0, x0, order, abstol)
Specialized method of `taylorstep` for jet transport applications.
Compute one-step Taylor integration for the ODE $\dot{x}=dx/dt=f(t, x)$
with initial condition $x(t_0)=x0$, returning the
time-step of the integration carried out and the updated value of `x0::TaylorN{T}`,
for `T<:Number`.
Here, `x0` is the initial (and returned) dependent variable, `order`
is the degree used for the `Taylor1` polynomials during the integration
and `abstol` is the absolute tolerance used to determine the time step
of the integration.
"""
function taylorstep{T<:Number}(f, t0::T, x0::TaylorN{T}, order::Int, abstol::T)
    # Initialize the Taylor1 expansions
    xT = Taylor1( x0, order )
    # Compute the Taylor coefficients
    jetcoeffs!(f, t0, xT)
    # Compute the step-size of the integration using `abstol`
    δt = stepsize(xT, abstol)
    x0 = evaluate(xT, δt)
    return δt, x0
end

doc"""
    taylorstep!(f, t0, x0, order, abstol)
Specialized method of `taylorstep!` for jet transport applications.
Compute one-step Taylor integration for the ODE $\dot{x}=dx/dt=f(t, x)$
with initial conditions $x(t_0)=x0$, a vector of type `TaylorN{T}`, returning the
step-size of the integration; `x0` is updated.
Here, `x0` is the initial (and updated) dependent variables, `order`
is the degree used for the `Taylor1` polynomials during the integration
and `abstol` is the absolute tolerance used to determine the time step
of the integration.
"""
function taylorstep!{T<:Number}(f, t0::T, x0::Array{TaylorN{T},1}, order::Int, abstol::T)
    # Initialize the vector of Taylor1 expansions
    xT = Array{Taylor1{TaylorN{T}}}(length(x0))
    for i in eachindex(x0)
        @inbounds xT[i] = Taylor1( x0[i], order )
    end
    # Compute the Taylor coefficients
    jetcoeffs!(f, t0, xT)
    # Compute the step-size of the integration using `abstol`
    δt = stepsize(xT, abstol)
    evaluate!(xT, δt, x0)
    return δt
end

doc"""
    taylorstep(f, t0, t1, x0, order, abstol)
Specialized method of `taylorstep` for jet transport applications.
Compute one-step Taylor integration for the ODE $\dot{x}=dx/dt=f(t, x)$
with initial conditions $x(t_0)=x0$, returning the
time-step of the integration carried out and the updated value of `x0::TaylorN{T}`,
with `T<:Number`.
Here, `x0` is the initial (and returned) dependent variables, `order`
is the degree used for the `Taylor1` polynomials during the integration
and `abstol` is the absolute tolerance used to determine the time step
of the integration. If the time step is larger than `t1-t0`, that difference
is used as the time step.
"""
function taylorstep{T<:Number}(f, t0::T, t1::T, x0::TaylorN{T}, order::Int, abstol::T)
    @assert t1 > t0
    # Initialize the Taylor1 expansions
    xT = Taylor1( x0, order )
    # Compute the Taylor coefficients
    jetcoeffs!(f, t0, xT)
    # Compute the step-size of the integration using `abstol`
    δt = stepsize(xT, abstol)
    if δt ≥ t1-t0
        δt = t1-t0
    end
    x0 = evaluate(xT, δt)
    return δt, x0
end

doc"""
    taylorstep!(f, t0, t1, x0, order, abstol)
Specialized method of `taylorstep!` for jet transport applications.
Compute one-step Taylor integration for the ODE $\dot{x}=dx/dt=f(t, x)$
with initial conditions $x(t_0)=x0$, a vector of type `TaylorN{T}`, returning the
step-size of the integration carried out and updating `x0`.
Here, `x0` is the initial (and updated) dependent variables, `order`
is the degree used for the `Taylor1` polynomials during the integration
and `abstol` is the absolute tolerance used to determine the time step
of the integration. If the time step is larger than `t1-t0`, that difference
is used as the time step.
"""
function taylorstep!{T<:Number}(f, t0::T, t1::T, x0::Array{TaylorN{T},1},
        order::Int, abstol::T)
    @assert t1 > t0
    # Initialize the vector of Taylor1 expansions
    xT = Array{Taylor1{TaylorN{T}}}(length(x0))
    for i in eachindex(x0)
        @inbounds xT[i] = Taylor1( x0[i], order )
    end
    # Compute the Taylor coefficients
    jetcoeffs!(f, t0, xT)
    # Compute the step-size of the integration using `abstol`
    δt = stepsize(xT, abstol)
    if δt ≥ t1-t0
        δt = t1-t0
    end
    evaluate!(xT, δt, x0)
    return δt
end

# taylorinteg
doc"""
    taylorinteg(f, x0, t0, tmax, order, abstol; keyword... )
Specialized method of `taylorinteg` for jet transport applications.
This is a jet transport Taylor integrator for the explicit ODE
$\dot{x}=f(t,x)$ with initial condition specified by `x0` at time `t0::T`.
Here, `x0` is either a `TaylorN{T}`, or an `Array{TaylorN{T},1}`.
It returns a vector with the values of time (independent variable),
and a vector (of type `typeof(x0)`) with the computed values of
the dependent variable(s). The integration stops when time
is larger than `tmax` (in which case the last returned values are
`t_max`, `x(t_max)`), or else when the number of saved steps is larger
than `maxsteps`.
The integrator uses polynomial expansions on the independent variable
of order `order`; the parameter `abstol` serves to define the
time step using the last two Taylor coefficients of the expansions.
The current keyword argument is `maxsteps=500`.
---
"""
function taylorinteg{T<:Number}(f, x0::TaylorN{T}, t0::T, tmax::T, order::Int,
        abstol::T; maxsteps::Int=500)

    # Allocation
    tv = Array{T}(maxsteps+1)
    xv = Array{TaylorN{T}}(maxsteps+1)

    # Initial conditions
    nsteps = 1
    @inbounds tv[1] = t0
    @inbounds xv[1] = x0

    # Integration
    while t0 < tmax
        xold = x0
        δt, x0 = taylorstep(f, t0, x0, order, abstol)
        if t0+δt ≥ tmax
            x0 = xold
            δt, x0 = taylorstep(f, t0, tmax, x0, order, abstol)
            t0 = tmax
            nsteps += 1
            @inbounds tv[nsteps] = t0
            @inbounds xv[nsteps] = x0
            break
        end
        t0 += δt
        nsteps += 1
        @inbounds tv[nsteps] = t0
        @inbounds xv[nsteps] = x0
        if nsteps > maxsteps
            warn("""
            Maximum number of integration steps reached; exiting.
            """)
            break
        end
    end

    #return tv, xv
    return view(tv,1:nsteps), view(xv,1:nsteps)
end

function taylorinteg{T<:Number}(f, q0::Array{TaylorN{T},1}, t0::T, tmax::T,
        order::Int, abstol::T; maxsteps::Int=500)

    # Allocation
    tv = Array{T}(maxsteps+1)
    dof = length(q0)
    xv = Array{TaylorN{T}}(dof, maxsteps+1)

    # Initial conditions
    @inbounds tv[1] = t0
    @inbounds xv[:,1] = q0[:]
    x0 = copy(q0)

    # Integration
    nsteps = 1
    while t0 < tmax
        xold = copy(x0)
        δt = taylorstep!(f, t0, x0, order, abstol)
        if t0+δt ≥ tmax
            x0 = xold
            δt = taylorstep!(f, t0, tmax, x0, order, abstol)
            t0 = tmax
            nsteps += 1
            @inbounds tv[nsteps] = t0
            @inbounds xv[:,nsteps] = x0[:]
            break
        end
        t0 += δt
        nsteps += 1
        @inbounds tv[nsteps] = t0
        @inbounds xv[:,nsteps] = x0[:]
        if nsteps > maxsteps
            warn("""
            Maximum number of integration steps reached; exiting.
            """)
            break
        end
    end

    return view(tv,1:nsteps), view(xv,:,1:nsteps)' #for xv, first do view, then transpose (otherwise it crashes)
end

# Integrate and return results evaluated at given time
doc"""
    taylorinteg(f, x0, t0, trange, order, abstol; keyword... )
Specialized method of `taylorinteg` for jet transport applications.
This is a jet transport Taylor integrator for the explicit ODE
$\dot{x}=f(t,x)$ with initial condition specified by `x0` at time `t0`.
Here, `x0` is either a `TaylorN{T}`, or an `Array{TaylorN{T},1}`.
It returns a vector with the values of time (independent variable),
and a vector (of type `typeof(x0)`) with the computed values of
the dependent variable(s), evaluated only at the times given by
`trange`. The integration stops when time
is larger than `tmax` (in which case the last returned values are
`t_max`, `x(t_max)`), or else when the number of saved steps is larger
than `maxsteps`.
The integrator uses polynomial expansions on the independent variable
of order `order`; the parameter `abstol` serves to define the
time step using the last two Taylor coefficients of the expansions.
The current keyword argument is `maxsteps=500`.
---
"""
function taylorinteg{T<:Number}(f, x0::TaylorN{T}, trange::Range{T},
        order::Int, abstol::T; maxsteps::Int=500)

    # Allocation
    nn = length(trange)
    xv = Array{TaylorN{T}}(nn)
    fill!(xv, TaylorN{T}(NaN))

    # Initial conditions
    @inbounds xv[1] = x0

    # Integration
    iter = 1
    while iter < nn
        t0, t1 = trange[iter], trange[iter+1]
        nsteps = 0
        while nsteps < maxsteps
            xold = x0
            δt, x0 = taylorstep(f, t0, x0, order, abstol)
            if t0+δt ≥ t1
                x0 = xold
                δt, x0 = taylorstep(f, t0, t1, x0, order, abstol)
                t0 = t1
                break
            end
            t0 += δt
            nsteps += 1
        end
        if nsteps ≥ maxsteps && t0 != t1
            warn("""
            Maximum number of integration steps reached; exiting.
            """)
            break
        end
        iter += 1
        @inbounds xv[iter] = x0
    end

    return xv
end

function taylorinteg{T<:Number}(f, q0::Array{TaylorN{T},1}, trange::Range{T},
        order::Int, abstol::T; maxsteps::Int=500)

    # Allocation
    nn = length(trange)
    dof = length(q0)
    x0 = similar(q0, TaylorN{T}, dof)
    fill!(x0, TaylorN{T}(NaN))
    xv = Array{eltype(q0)}(dof, nn)
    @inbounds for ind in 1:nn
        xv[:,ind] = x0[:]
    end

    # Initial conditions
    @inbounds x0[:] = q0[:]
    @inbounds xv[:,1] = q0[:]

    # Integration
    iter = 1
    while iter < nn
        t0, t1 = trange[iter], trange[iter+1]
        nsteps = 0
        while nsteps < maxsteps
            xold = copy(x0)
            δt = taylorstep!(f, t0, x0, order, abstol)
            if t0+δt ≥ t1
                x0 = xold
                δt = taylorstep!(f, t0, t1, x0, order, abstol)
                t0 = t1
                break
            end
            t0 += δt
            nsteps += 1
        end
        if nsteps ≥ maxsteps && t0 != t1
            warn("""
            Maximum number of integration steps reached; exiting.
            """)
            break
        end
        iter += 1
        @inbounds xv[:,iter] = x0[:]
    end

    return xv'
end

taylorinteg (generic function with 4 methods)

In [3]:
function undrivenHelium(τ, q)
    tT, Q₁, Q₂, P₁, P₂ = q
    
    t1 = Q₁^2
    t2 = Q₂^2
    t = t1 * t2
    R12 = t1 - t2
    @show 1/R12
    q₁ = 0.25*t2*P₁
    p₁ = 2Q₁*(-0.125P₂^2 + Z  - t2 - t2/((R12)^2)^(1/2) + t*R12/((R12)^2)^(3/2))
    q₂ = 0.25*t1*P₂
    p₂ = 2Q₂*(-0.125P₁^2 + Z  - t1 - t1/((R12)^2)^(1/2) - t*R12/((R12)^2)^(3/2))

    return [t, q₁, q₂, p₁, p₂] 

end
undrivenHelium(v) = undrivenHelium(zero(eltype(v)), v)


var1 = set_variables("t q1 q2 p1 p2", order = 1)


function condini(x10::Float64, px10::Float64)
    @assert x10 != 0
    Q1 = sqrt(x10)
    Q2 = 0.0
    P1 = px10*sqrt(x10)    
    P2 = sqrt(8Z)    
    return [0, Q1, Q2, P1, P2]
end

function regHam(t, Q₁, Q₂, P₁, P₂)
    #Cantidades auxiliares
    P₁² = P₁^2
    P₂² = P₂^2
    Q₁² = Q₁^2
    Q₂² = Q₂^2
    nf = abs(Q₁² - Q₂²)
    
    H = 0.125*(P₁²*Q₂² + P₂²*Q₁²) - Z*(Q₁² + Q₂²) + Q₁²*Q₂²*(1.0 + 1.0/nf)
    return H
end


regHam(v) = regHam(v...)

function errHam(N::Int)
    srand(487293456)
    J = [0 0 0 0 0; 0 0 0 1 0; 0 0 0 0 1; 0 -1 0 0 0; 0 0 -1 0 0 ]
    
    dnorm = zeros(N)
    als = typeof(zeros(5))[]
    for j in 1:N
        al = 10*rand(5)
        #al = [BigFloat(x) for x in al1]
        alt = al + var1
        ene = J*∇(regHam(alt))
        ene1 = [ene[k].coeffs[1].coeffs[1] for k in 1:5]
        meq = undrivenHelium(al)
        push!(als, al)
        meq[1] = 0
        dnorm[j] = norm(meq - ene1) 
    end
    return dnorm, als 
end

"""
    stabilitymatrix!{T<:Number, S<:Number}(eqsdiff, t0::T, x::Array{T,1},
        jjac::Array{T,2})
    stabilitymatrix!{T<:Number, S<:Number}(eqsdiff, t0::T, x::Array{Taylor1{T},1},
        jjac::Array{Taylor1{T},2})
Updates the matrix `jjac` (linearized equations of motion)
computed from the equations of motion (`eqsdiff`), at time `t0`
at `x0`.
"""
function stabilitymatrix!{T<:Number}(eqsdiff, t0::T, x::Array{T,1},
        jjac::Array{T,2})
    δx = Array{TaylorN{T}}( length(x) )
    @inbounds for ind in eachindex(x)
        δx[ind] = x[ind] + TaylorN(T,ind,order=1)
    end
    jjac[:] = jacobian( eqsdiff(t0, δx), x )
    nothing
end

stabilitymatrix!

In [4]:
function q2sign(xold, x0, tol)
    if abs(xold[3]) < tol
        sq2o = 0
    else
        sq2o = sign(xold[3])
    end
    
    if abs(x0[3]) < tol
        sq2 = 0
    else
        sq2 = sign(x0[3])
    end
    if abs(xold[5]) < tol
        sp2o = 0
    else
        sp2o = sign(xold[5])
    end
    
    if abs(x0[5]) < tol
        sp2 = 0
    else
        sp2 = sign(x0[5])
    end
    return sq2o, sq2, sp2o, sp2
end
    

q2sign (generic function with 1 method)

In [5]:
a1 = condini(rand(2)...)

5-element Array{Float64,1}:
 0.0    
 0.84598
 0.0    
 0.78209
 4.0    

In [6]:
expe1 = a1 + var1

5-element Array{TaylorSeries.TaylorN{Float64},1}:
                        1.0 t + 𝒪(‖x‖²)
  0.8459797304085561 + 1.0 q1 + 𝒪(‖x‖²)
                       1.0 q2 + 𝒪(‖x‖²)
  0.7820902133103941 + 1.0 p1 + 𝒪(‖x‖²)
                 4.0 + 1.0 p2 + 𝒪(‖x‖²)

In [21]:
Taylor1(expe1[3].coeffs[1:1])

  0.0 + 𝒪(t¹)

In [25]:
eT1 = (typeof(similar(expe1)))[Taylor1(expe1[1].coeffs[1:1]), Taylor1(expe1[2].coeffs[1:1]), 
    Taylor1(expe1[3].coeffs[1:1]), Taylor1(expe1[4].coeffs[1:1]), Taylor1(expe1[5].coeffs[1:1])]
undrivenHelium(0.0, eT1)

LoadError: MethodError: Cannot `convert` an object of type TaylorSeries.Taylor1{TaylorSeries.HomogeneousPolynomial{Float64}} to an object of type Array{TaylorSeries.TaylorN{Float64},1}
This may have arisen from a call to the constructor Array{TaylorSeries.TaylorN{Float64},1}(...),
since type constructors fall back to convert methods.

In [18]:
taylorstep!(undrivenHelium, 0.0, 10.0, expe1, order, abs_tol)

ord = 1
a1 =   1.0 t + 𝒪(‖x‖²) + 𝒪(t¹)
a1 =   0.8459797304085561 + 1.0 q1 + 𝒪(‖x‖²) + 𝒪(t¹)
a1 =   1.0 q2 + 𝒪(‖x‖²) + 𝒪(t¹)
a1 =   0.7820902133103941 + 1.0 p1 + 𝒪(‖x‖²) + 𝒪(t¹)
a1 =   4.0 + 1.0 p2 + 𝒪(‖x‖²) + 𝒪(t¹)
xaux = TaylorSeries.Taylor1{TaylorSeries.TaylorN{Float64}}[  1.0 t + 𝒪(‖x‖²) + 𝒪(t¹),  0.8459797304085561 + 1.0 q1 + 𝒪(‖x‖²) + 𝒪(t¹),  1.0 q2 + 𝒪(‖x‖²) + 𝒪(t¹),  0.7820902133103941 + 1.0 p1 + 𝒪(‖x‖²) + 𝒪(t¹),  4.0 + 1.0 p2 + 𝒪(‖x‖²) + 𝒪(t¹)]


LoadError: MethodError: no method matching isinf(::TaylorSeries.TaylorN{Float64})[0m
Closest candidates are:
  isinf([1m[31m::BigFloat[0m) at mpfr.jl:792
  isinf([1m[31m::Float16[0m) at float16.jl:118
  isinf([1m[31m::FixedPointNumbers.Normed{T<:Unsigned,f}[0m) at /home/daniel/.julia/v0.5/FixedPointNumbers/src/normed.jl:113
  ...[0m

In [ ]:
function vareqs(t, q, A)
    qn = A * q
    return qn
end

stabmat = eye(5)
stabmat[1, 1] = 0.0

varuh(t, q) = vareqs(t, q, stabmat)

In [ ]:
a1 = condini(rand(2)...)
expe1 = var1 + a1

In [ ]:
varuh(0.0, var1)

In [ ]:
var1[2].coeffs[2].coeffs

In [ ]:
typeof(var1)

In [ ]:
###Implementamos nuevo código 
function taylorstep1!{T<:Number}(f, t0::T, t1::T, x0::Array{T,1},
        order::Int, abstol::T)
    @assert t1 > t0
    # Initialize the vector of Taylor1 expansions
    xT = Array{Taylor1{TaylorN{T}}}(length(x0))
    for i in eachindex(x0)
        @inbounds xT[i] = Taylor1( x0[i], order )
    end
    # Compute the Taylor coefficients
    jetcoeffs!(f, t0, xT)
    # Compute the step-size of the integration using `abstol`
    δt = TaylorIntegration.stepsize(xT, abstol)
    if δt ≥ t1-t0
        δt = t1-t0
    end
    evaluate!(xT, δt, x0)
    return δt, xT
end


function mytaylorinteg{T<:Number}(f, q0::Array{TaylorN{T},1}, t0::T, tmax::T, order::Int,
    abstol::T; tsteps::Int = 1000, tol::Float64=1e-20, maxsteps::Int=5000)

    # Allocation
    tv = Array{T}(maxsteps)
    dof = length(q0)
    xv = Array{T}(dof, maxsteps)
    δs = Array{TaylorN}(maxsteps)
    sm0 = eye(dof)
    
    function vareqs(t, q, A)
        qn = A * q
        return qn
    end
    varuh(t, q) = vareqs(t, q, sm0)
    
    
    # Initial conditions
    @inbounds tv[1] = t0
    @inbounds xv[:,1] = q0[:]
    @inbounds δs[1] = var1
    x0 = copy(q0)
    δ0 = copy(δs[1])
    sum1 = q0[3]/maxsteps

    # Integration
    nsteps = 1
    δtn = Inf
    while t0 < tmax
        xold = copy(x0)
        δold = copy(δ0)
        δt, sT = taylorstep1!(f, t0, tmax, x0, δ0, order, abstol)
        sq2o, sq2, sp2o, sp2 = q2sign(xold, x0, tol)
        steps1 = 0
        x00 = copy(x0)
        δ00 = copy(δ0)
        bool1 = false
        
        while sq2o*sq2 == -1 || (sp2o*sp2 == -1 && abs(xold[3]) < tol)
            bool1 = true
            q2T = sT[3]
            dq2T = derivative(q2T)
            δtn = copy(δt)
            for nc in 1:20
                δtn = δtn - evaluate(q2T, δtn)/evaluate(dq2T, δtn)
            end
            evaluate!()
            evaluate!(sT, δtn, x0)
            sq2o, sq2, sp2o, sp2 = q2sign(xold, x0, tol)
            steps1 += 1
            if steps1  ≥ tsteps
                break
            end
        end
                
        if nsteps ≥ maxsteps 
            warn("""
            Maximum number of integration steps reached; exiting.
            """)
            break
        end
        
                
        nsteps += 1
        
        if bool1 == true
            @inbounds tv[nsteps] = t0 + δtn
            @inbounds xv[:,nsteps] = x0[:]
            stabilitymatrix!(undrivenHelium, t0 + δtn, x0, sm0)
            jjac0 = sm0 * arrjac[nsteps - 1]
            arrjac[nsteps] = jjac0
            x0 = x00
            nsteps += 1
            stabilitymatrix!(undrivenHelium, t0 + δt, x0, sm0)
            jjac0 = sm0 * arrjac[nsteps - 2]
            arrjac[nsteps] = jjac0
        else
            stabilitymatrix!(undrivenHelium, t0 + δt, x0, sm0)
            jjac0 = sm0 * arrjac[nsteps - 1]
            arrjac[nsteps] = jjac0            
        end
        t0 += δt
        @inbounds tv[nsteps] = t0
        @inbounds xv[:,nsteps] = x0[:]
        sum1 += x0[3]/maxsteps
            
    end

    return view(tv,1:nsteps), view(xv,:,1:nsteps)', arrjac[1:nsteps]
end

In [ ]:
c1 = condini(2.0, 0.0) + var1

In [ ]:
@time t1, x1 = taylorinteg(undrivenHelium, c1, t0, 2.0, order, abs_tol, maxsteps=100_000);

In [ ]:
@time t2, x2, m1 = mytaylorinteg(undrivenHelium, c1, t0, 2.0, order, abs_tol, maxsteps=100)

In [ ]:
m1[64]

In [ ]:
for j in 1:64
    a = (m1[j][2:end,2:end])
    println(a)
end